In [106]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import unicodedata
from nltk.stem import WordNetLemmatizer

In [ ]:
# full_raw = pd.read_csv('/Users/javier/VSCODE/datahub/enron_data_fraud_labeled.csv') 

# filtered_raw = full_raw['Body'].iloc[:100_000]

# filtered_raw.to_csv('test.csv')

In [ ]:
filtered_raw = pd.read_csv('/Users/javier/VSCODE/local/DSA4213_vsc/final_project/test.csv')
filtered_raw= filtered_raw.drop(columns=['Unnamed: 0'])

In [85]:
test = "Hi John,  Please check out our new product at https://www.example.com/special-offer.  You can also visit www.testsite.org for more details.  I've attached the latest report as quarterly_results.pdf and also a backup copy as report.docx send to abc@gmail.com and john.doe@gmail.org. Let me know if you have trouble opening summary.xlsx.  Best,  Alice"

html_text = """
<html>
  <head>
    <title>Quarterly Update</title>
  </head>
  <body>
    <h1>Special Offer!</h1>
    <p>Dear customer,</p>
    
    <p>
      Please download the latest reports:
      <a href="https://example.com/files/quarterly_report.pdf">Quarterly Report</a>,
      <a href="https://example.com/files/summary.docx">Summary</a>,
      and <a href="https://example.com/files/data.xlsx">Data File</a>.
    </p>

    <p>
      If you cannot access the files, please email 
      <a href="mailto:support@example.com">support@example.com</a>.
    </p>

    <p>
      Alternatively, you may contact John at john.doe@workmail.org or visit our site 
      <a href="http://www.testsite.org">www.testsite.org</a>.
    </p>

    <p>
      Attached reference documents: <b>budget_2024.pdf</b>, <b>plan_final.docx</b>
    </p>

    <p>
      loveeeeeeeeee
      lovee33333eeee
      a-p-p-l-e
      b.a.n.a.n.a
      fr33 c4$h 
      Hello🌍!! This*** is a test 😎 #spam @user $100...
      45 46 20000 32323 $222.22
    </p
  </body>
</html>
"""


In [160]:
# masking special token
def mask_tokens(text):
    # replace URLs (http, https, www)
    text = re.sub(r'(https?://\S+|www\.\S+)', '<URL>', text)

    # replace common file extensions (customize list)
    text = re.sub(r'\b[\w\-]+\.(pdf|docx|xlsx|txt|csv|tar|doc\.gz|doc)\b', '<FILE>', text)

    # emails
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '<EMAIL>', text)

    # money 
    text = re.sub(r'\$\d+(?:\.\d{2})?','<MONEY>',text)

    # numbers 
    text = re.sub(r'\b\d+\b','<NUMBER>',text)
    text = text.replace('<NUMBER>', '')

    return text


# un HTML raw text 
def strip_html(raw_html):
    """
    Strip HTML tags, scripts, styles, and normalize whitespace
    to return clean raw text from HTML emails.
    """
    soup = BeautifulSoup(raw_html, "html.parser")

    
    for a in soup.find_all("a"):
        a_attribute = a['href'] 

        a_attribute = mask_tokens(a_attribute)

        if a_attribute == '<URL>' : 
            a.replace_with('<URL>')

        elif a_attribute =='<EMAIL>' : 
            a.replace_with('<EMAIL>')
        
        elif a_attribute == '<FILE>' : 
            a.replace_with('<FILE>')

        elif a_attribute == '<MONEY>' : 
            a.replace_with('<MONEY>')
        
        elif a_attribute == '<NUMBER>' : 
            a.replace_with('<NUMBER>')

    # remove script, style, head, and metadata tags
    for tag in soup(["script", "style", "head", "title", "meta", "[document]"]):
        tag.decompose()

    # extract text
    text = soup.get_text(separator=" ")

    # normalize unicode 
    text = unicodedata.normalize("NFKC", text)

    # replace non-breaking spaces specifically (unicode)
    text = text.replace("\xa0", " ")

    # collapse all whitespace tokens (line breaks, tabs, multiple spaces) into one space and remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    # rim leading/trailing spaces
    return text

# special case handling
mapper = str.maketrans({
    '0':'o','1':'l','3':'e','4':'a','5':'s','7':'t','$':'s','@':'a'
})

def deobfuscate_words(text):
    # replace text to number 
    text = text.translate(mapper)
    # remove weird spaces etc 
    text = re.sub(r'(?i)(?<=\w)[^A-Za-z0-9\s]{1,3}(?=\w)', '', text)
    return text

def word_capper(text):
    text = re.sub(r'(.)\1{' + str(2) + r',}', lambda m: m.group(1)*2, text)
    text = re.sub(r'([!?.,])\1{1,}', r'\1\1', text)
    return text


# whitelist filtering
def char_lvl_whitelist_filter(text): 
    text = re.sub(r'[^a-zA-Z0-9\s\.\,\!\?\'\":;\-\_\(\)\@\#\$\%\^\&\<\>]', '', text)
    return text

# word level processor 
def lemmatizer(text) :
    lemmatizer = WordNetLemmatizer()
    split = []
    sentence = ''

    text_split = text.split(' ')

    for word in text_split : 
        lemmatizer.lemmatize(word)
        split.append(word)
        
        sentence.join(word)

    return sentence


    

In [96]:
raw = html_text
raw = strip_html(raw) # process html first to capture links from <a> tags
raw = mask_tokens(raw) # mask special tokens 
raw = deobfuscate_words(raw)
raw = word_capper(raw)
raw = char_lvl_whitelist_filter(raw)

print(raw)



Special Offer! Dear customer, Please download the latest reports: <URL> , <URL> , and <URL> . If you cannot access the files, please email <EMAIL> . Alternatively, you may contact John at <EMAIL> or visit our site <URL> . Attached reference documents: <FILE> , <FILE> lovee lovee apple banana free cash Hello!! This is a test  #spam auser <MONEY>.. <NUMBER> <NUMBER> <NUMBER> <NUMBER> <MONEY>


In [ ]:
def preprocess_email_text(
    input_df
): 
    """
    input : dataframe with only text column 
    output : dataframe with cleaned sentences
    """
    # loop every row 
    for raw in input_df :
        
        
        raw = strip_html(raw) # process html first to capture links from <a> tags
        raw = mask_tokens(raw) # mask special tokens 
        raw = deobfuscate_words(raw)
        raw = word_capper(raw)
        raw = lemmatizer(raw)
        raw = char_lvl_whitelist_filter(raw)
        # print(raw)

    return input_df
    



In [ ]:
debug_test = filtered_raw['Body'].iloc[:1]


0    Status John: I'm not really sure what happened...
Name: Body, dtype: object


In [164]:
for x in debug_test: 
    print(x)

res = preprocess_email_text(debug_test)


for x in res: 
    print(x)


Status John: I'm not really sure what happened between us.? I was under the impression after my visit to Houston that we were about to enter into a trial agreement for my advisory work.? Somehow,?this never occurred.? Did I say or do something wrong to screw this up??? I don't know if you've blown this whole thing off, but I still hope you are interested in trying?to create an arrangement.? As a courtesy, here is my report from this past weekend.? If you are no longer interested in my work, please tell me so.??Best wishes, Mark Sagel Psytech Analytics (410)308-0245? energy2000-1112.doc
Status John: I'm not really sure what happened between us.? I was under the impression after my visit to Houston that we were about to enter into a trial agreement for my advisory work.? Somehow,?this never occurred.? Did I say or do something wrong to screw this up??? I don't know if you've blown this whole thing off, but I still hope you are interested in trying?to create an arrangement.? As a courtesy